# Projetando um PID

## Modelando o sistema

Este trabalho foi baseado no problema 3.17 do livro *Control Systems* de *Norman S. Nise*, Sétima edição.

![Modelando o vôo do míssil](img/missil.png)

O problema modela o vôo de um míssil, que está sujeito a quatro forças: empuxo (*thrust*), sustentação (*lift*),
arrasto (*drag*) e gravidade. O míssil voa com um ângulo de ataque, $\alpha$, do seu eixo longitudinal, criando sustentação. Para seguir um determinado rumo, o ângulo do corpo da vertical, $\phi$, é controlado rotacionando o motor na cauda. 

A função de transferência relaciona o ângulo do corpo, $\phi$, e sua posição angular, $\delta$, do motor na forma, como mostrado na equação abaixo:

\begin{align}
 \frac{\Phi(s)}{\delta(s)} = \frac{K_a s + K_b}{K_3 s^3 + K_2 s^2 + K_1 s + K_0} \label{eq:plant}
\end{align}

## Entendendo a planta

Desejamos projetar um controlador PID para controlar o míssil, conforme relacionado pelo diagrama de blocos abaixo:

![Diagrama de blocos](img/block-diagram.svg)

Para fins práticos, escolheremos os parâmetros da planta de forma arbitrária.

A resposta ao degrau e impulso da planta são mostrados abaixo:

In [6]:
%matplotlib ipympl
import matplotlib.pyplot as plt 
from control import (TransferFunction, step_response, bode_plot,
                     impulse_response, series, feedback, rlocus)

ka, kb,  = [1, 5]
k3, k2, k1, k0 = [2, 50, 10, 1]

missile_tf = TransferFunction([ka, kb], [k3, k2, k1, k0])

In [7]:
def plot_step_response(tf, title='Step response'):
    time, output = step_response(tf)
    fig = plt.figure()
    plt.plot(time, output)
    plt.title(title)
    plt.show()
    
plot_step_response(missile_tf, title='Plant step response')

FigureCanvasNbAgg()

E também podemos visualizar a resposta em frequência da planta através do diagrama de *Bode*:

In [8]:
plt.figure()
mag, phase, omega = bode_plot(missile_tf)

FigureCanvasNbAgg()

## Ajustando controlador

### Método resposta a frequência de Ziegler-Nichols
A fim de aplicarmos este método, acharemos o ganho crítico do sistema. Para isso analisaremos o lugar das raízes:

In [10]:
%matplotlib ipympl
loci = rlocus(missile_tf,Plot=True, PrintGain=True)

FigureCanvasNbAgg()

Porém, percebe-se que nosso sistema não tem um ganho crítico, podemos aumentá-lo indefinidamente. Logo, não podemos utilizar este método. 
Recorreremos a outro método.

### Método resposta ao degrau de Ziegler-Nichols